In [1]:
import requests
import os
import pandas as pd

 - Filter for electronic devices

 - P31 is the property for "instance of"

 - P279 is the property for "subclass of"

 - Q71266741 is the item for "smartphone"

 - mobiled phone has the following ID: Q17517

 - Q16338 for personal computer 
 - ?device wdt:P31/wdt:P279*  ?subclass.

 - ?subclass wdt:P279 ?metaclass .

 - ?metaclass wdt:P642 wd:Q68 .



In [2]:
# Define the properties we want to retrieve
# properties = ['inception date', 'mass', 'part of the series', 'subclass of', 'instance of', 'color', 'manufacturer', 'CPU', 'memory capacity', 'max size', 'energy storage capacity', 'operating system', 'height', 'width', 'thickness', 'display technology', 'price']

# Construct the query string
query = """
SELECT ?device ?deviceLabel ?colorLabel  ?inception ?precisionLabel ?instance_ofLabel ?height ?unitHeightLabel ?width ?unitWidthLabel ?weight ?unitWeightLabel ?price ?unitPriceLabel
WHERE
{
  ?device wdt:P31/wdt:P279* [p:P279*/pq:P642*[wdt:P279* wd:Q68]].

  OPTIONAL {?device wdt:P31 ?instance_of.}
  OPTIONAL {?device wdt:P462 ?color.}
  OPTIONAL {?device p:P2284/psv:P2284 [ wikibase:quantityAmount     ?price ;
                                        wikibase:quantityUnit       ?unitPrice].}
                                        
  OPTIONAL {?device p:P2048/psv:P2048 [ wikibase:quantityAmount     ?height ;
                                        wikibase:quantityUnit       ?unitHeight].}
  OPTIONAL {?device p:P2049/psv:P2049 [ wikibase:quantityAmount     ?width ;
                                        wikibase:quantityUnit       ?unitWidth ].}
  OPTIONAL {?device p:P2067/psv:P2067 [ wikibase:quantityAmount     ?weight ;
                                        wikibase:quantityUnit       ?unitWeight].}  
  OPTIONAL {?device p:P571/psv:P571 ?timenode .
            ?timenode wikibase:timeValue         ?inception.
            ?timenode wikibase:timePrecision     ?timeprecision.
            # get the label of time precision
            {SELECT ?precision (xsd:integer(?precisionDecimal) AS ?timeprecision) {
              ?precision  wdt:P2803  ?precisionDecimal .}
            }
          }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
"""

# Define the endpoint URL for the Wikidata SPARQL API
url = "https://query.wikidata.org/sparql"

# Define the headers for the request
headers = {"Accept": "application/sparql-results+json"}

# Send the request and get the response
response = requests.get(url, headers=headers, params={"query": query, "timeout": "600000000"})

# Parse the response into a pandas dataframe
data = pd.read_json(response.content.decode())["results"]["bindings"]
df_full = pd.json_normalize(data)
df_full.fillna(value="<na>", inplace=True)
df_full

,device.type,device.value,inception.datatype,inception.type,inception.value,deviceLabel.xml:lang,deviceLabel.type,deviceLabel.value,precisionLabel.xml:lang,precisionLabel.type,...,unitWeightLabel.value,price.datatype,price.type,price.value,unitPriceLabel.xml:lang,unitPriceLabel.type,unitPriceLabel.value,colorLabel.xml:lang,colorLabel.type,colorLabel.value
0,uri,http://www.wikidata.org/entity/Q42266831,http://www.w3.org/2001/XMLSchema#dateTime,literal,2017-01-01T00:00:00Z,en,literal,Nokia 7,en,literal,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
1,uri,http://www.wikidata.org/entity/Q42266831,http://www.w3.org/2001/XMLSchema#dateTime,literal,2017-01-01T00:00:00Z,en,literal,Nokia 7,en,literal,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
2,uri,http://www.wikidata.org/entity/Q42266831,http://www.w3.org/2001/XMLSchema#dateTime,literal,2017-01-01T00:00:00Z,en,literal,Nokia 7,en,literal,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
3,uri,http://www.wikidata.org/entity/Q42266831,http://www.w3.org/2001/XMLSchema#dateTime,literal,2017-01-01T00:00:00Z,en,literal,Nokia 7,en,literal,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
4,uri,http://www.wikidata.org/entity/Q42266831,http://www.w3.org/2001/XMLSchema#dateTime,literal,2017-01-01T00:00:00Z,en,literal,Nokia 7,en,literal,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128543,uri,http://www.wikidata.org/entity/Q117089235,<na>,<na>,<na>,en,literal,Chameleon,<na>,<na>,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
128544,uri,http://www.wikidata.org/entity/Q117089271,<na>,<na>,<na>,en,literal,zope.contentprovider,<na>,<na>,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
128545,uri,http://www.wikidata.org/entity/Q117089310,<na>,<na>,<na>,en,literal,z3c.ptcompat,<na>,<na>,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>
128546,uri,http://www.wikidata.org/entity/Q117105107,<na>,<na>,<na>,<na>,literal,Q117105107,<na>,<na>,...,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>,<na>


In [3]:
all(["Q" in item["device"]["value"] for item in data])

True

In [4]:
[item["device"] for item in data if item["device"]["value"].endswith("Q49046")]

[{'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'},
 {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q49046'}]

In [5]:
# group by deviceLabel and keep the whole set
df = df_full.groupby('deviceLabel.value').agg(lambda x: list(set(x))).reset_index()
# select only relevant columns
df = df[["device.value", 'deviceLabel.value', 'colorLabel.value', 'inception.value', 'precisionLabel.value', 'instance_ofLabel.value', 'height.value', 'unitHeightLabel.value', 'width.value', 'unitWidthLabel.value', 'weight.value', 'unitWeightLabel.value', 'price.value', 'unitPriceLabel.value']]
# rename columns: simplify and make consistent snake_case with main_unit naming scheme like height_unit
df.columns = ["ID", 'device', 'color', 'inception', 'precision', 'instance_ofLabel', 'height', 'height_unit', 'width', 'width_unit', 'weight', 'weight_unit', 'price', 'price_unit']
df

,ID,device,color,inception,precision,instance_ofLabel,height,height_unit,width,width_unit,weight,weight_unit,price,price_unit
0,[http://www.wikidata.org/entity/Q30015097],"""BeeZy"" BG-1000",[<na>],[1978-01-01T00:00:00Z],[year],[computer],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
1,[http://www.wikidata.org/entity/Q72908471],"""PARAM Yuva"" Cluster",[<na>],[<na>],[<na>],[supercomputer],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
2,[http://www.wikidata.org/entity/Q74585009],1,[<na>],[<na>],[<na>],[supercomputer],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
3,[http://www.wikidata.org/entity/Q161255],1&1 SmartPad,[<na>],[<na>],[<na>],[tablet computer],[196.6],[millimetre],[130],[millimetre],[470],[gram],[<na>],[<na>]
4,[http://www.wikidata.org/entity/Q97994612],1.8-inch drive,[<na>],[<na>],[<na>],[drive form factor],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13105,[http://www.wikidata.org/entity/Q107386742],zvmcloudconnector,[<na>],[<na>],[<na>],[Python library],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
13106,[http://www.wikidata.org/entity/Q107386259],zxcvbn,[<na>],[<na>],[<na>],[Python library],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
13107,[http://www.wikidata.org/entity/Q290639],Étoilé,[<na>],[<na>],[<na>],[desktop environment],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
13108,[http://www.wikidata.org/entity/Q73403816],Świerk Computing Centre,[<na>],[<na>],[<na>],[supercomputer],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]


In [6]:
df[df["device"].apply(lambda x: x.lower()).str.contains("iphone")]

,ID,device,color,inception,precision,instance_ofLabel,height,height_unit,width,width_unit,weight,weight_unit,price,price_unit
9606,[http://www.wikidata.org/entity/Q621427],iPhone,[black],[2007-01-09T00:00:00Z],[day],[smartphone model],[4.5],[inch],[2.4],[inch],"[4.8, 135]","[ounce, gram]",[<na>],[<na>]
9607,[http://www.wikidata.org/entity/Q100332903],iPhone 12,[<na>],[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
9608,[http://www.wikidata.org/entity/Q108118280],iPhone 13,"[white, Product Red, pink, blue, black]",[2021-09-14T00:00:00Z],[day],[smartphone model],[146.7],[millimetre],[71.5],[millimetre],[173],[gram],"[1029, 909, 1259]",[euro]
9609,[http://www.wikidata.org/entity/Q108541719],iPhone 13 Pro,"[silver, graphite, gold, blue]",[2021-09-14T00:00:00Z],[day],[smartphone model],[146.7],[millimetre],[71.5],[millimetre],[204],[gram],[<na>],[<na>]
9610,[http://www.wikidata.org/entity/Q108541741],iPhone 13 Pro Max,"[silver, graphite, gold, blue]",[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
9611,[http://www.wikidata.org/entity/Q108541666],iPhone 13 mini,[<na>],[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
9612,[http://www.wikidata.org/entity/Q110397828],iPhone 14,"[midnight blue, starlight, Product Red, blue, ...",[<na>],[<na>],[smartphone model],[146.7],[millimetre],[71.5],[millimetre],[172],[gram],[799],[United States dollar]
9613,[http://www.wikidata.org/entity/Q113816207],iPhone 14 Plus,[<na>],[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[899],[United States dollar]
9614,[http://www.wikidata.org/entity/Q113800722],iPhone 14 Pro,"[silver, gold, violet]",[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]
9615,[http://www.wikidata.org/entity/Q113816991],iPhone 14 Pro Max,[<na>],[<na>],[<na>],[smartphone model],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>],[<na>]


In [7]:
sdf = df[df["height"].apply(lambda x: x[0] if x[0]!="<nan>" else (x[1] if len(x)>1 else "<nan>")) != "<na>"]
sdf

,ID,device,color,inception,precision,instance_ofLabel,height,height_unit,width,width_unit,weight,weight_unit,price,price_unit
3,[http://www.wikidata.org/entity/Q161255],1&1 SmartPad,[<na>],[<na>],[<na>],[tablet computer],[196.6],[millimetre],[130],[millimetre],[470],[gram],[<na>],[<na>]
229,[http://www.wikidata.org/entity/Q15261471],Alcatel One Touch Fire,[<na>],[<na>],[<na>],[smartphone model],[115],[millimetre],[62],[millimetre],[108],[gram],[<na>],[<na>]
246,[http://www.wikidata.org/entity/Q24465055],Alcatel Pop C1,[<na>],[<na>],[<na>],[smartphone model],[112.5],[millimetre],[62],[millimetre],[100],[gram],[<na>],[<na>]
303,[http://www.wikidata.org/entity/Q4748871],Amstrad Mega PC,[<na>],[<na>],[<na>],"[home video game console model, computer model]",[78],[millimetre],[325],[millimetre],[<na>],[<na>],[<na>],[<na>]
376,[http://www.wikidata.org/entity/Q16528397],Archos 50 Platinum,[<na>],[<na>],[<na>],[smartphone model],[8.9],[millimetre],[72.4],[millimetre],[160.2],[gram],[<na>],[<na>]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9606,[http://www.wikidata.org/entity/Q110397828],iPhone 14,"[blue, purple, Product Red, midnight blue, sta...",[<na>],[<na>],[smartphone model],[146.7],[millimetre],[71.5],[millimetre],[172],[gram],[799],[United States dollar]
9610,[http://www.wikidata.org/entity/Q300859],iPhone 3GS,"[white, black]",[2009-06-08T00:00:00Z],[day],[smartphone model],[115.5],[millimetre],[62.1],[millimetre],[135],[gram],[<na>],[<na>]
9612,[http://www.wikidata.org/entity/Q20966885],iPhone 6S Plus,[<na>],[2015-09-09T00:00:00Z],[day],[smartphone model],[158.2],[millimetre],[77.9],[millimetre],[192],[gram],[<na>],[<na>]
9613,[http://www.wikidata.org/entity/Q26831164],iPhone 7,"[rose gold, gold, red, black, silver]",[2016-09-07T00:00:00Z],[day],[smartphone],[138.3],[millimetre],[67.1],[millimetre],[138],[gram],"[989, 20500, 879, 769, 24200]","[euro, baht]"


# Import individual subclasses

In [11]:
from qwikidata.sparql import get_subclasses_of_item, return_sparql_query_results
subclasses = get_subclasses_of_item("Q68")

In [21]:
query = """
SELECT DISTINCT ?item ?itemLabel ?classLabel
WHERE {
  ?item wdt:P31/wdt:P279* ?class .
  ?class wdt:P279* ?ancestor .
  ?ancestor p:P279 ?qualifierStatement .
  ?qualifierStatement pq:P642 ?subClassOfQ68 .
  ?subClassOfQ68 wdt:P279* wd:Q68 .
SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
"""
res = return_sparql_query_results(query)
df = pd.json_normalize(res["results"]["bindings"])
df = df[["item.value", "itemLabel.value", "classLabel.value"]]
df.columns = ["item", "itemLabel", "classLabel"]
df.head()

,item,itemLabel,classLabel
0,http://www.wikidata.org/entity/Q7517814,Simile,computer virus
1,http://www.wikidata.org/entity/Q8073103,Zmist,computer virus
2,http://www.wikidata.org/entity/Q110662443,FlorisBoard,computer keyboard
3,http://www.wikidata.org/entity/Q2124587,RST.b,computer virus
4,http://www.wikidata.org/entity/Q4669447,Abraxas,computer virus


In [22]:
df[df.itemLabel.str.contains("iPhone")]

,item,itemLabel,classLabel
1434,http://www.wikidata.org/entity/Q300859,iPhone 3GS,smartphone model
1560,http://www.wikidata.org/entity/Q621427,iPhone,smartphone model
2939,http://www.wikidata.org/entity/Q621427,iPhone,computer model
2994,http://www.wikidata.org/entity/Q300859,iPhone 3GS,computer model
3811,http://www.wikidata.org/entity/Q20966885,iPhone 6S Plus,smartphone model
4090,http://www.wikidata.org/entity/Q23309966,iPhone SE (1st generation),smartphone model
4361,http://www.wikidata.org/entity/Q20966885,iPhone 6S Plus,computer model
4398,http://www.wikidata.org/entity/Q23309966,iPhone SE (1st generation),computer model
4658,http://www.wikidata.org/entity/Q66816688,"iPhone 5s, model A1533",computer model
5278,http://www.wikidata.org/entity/Q100332903,iPhone 12,computer model
